# TTS MODEL




#1.   Installation of libraries







In [1]:
!pip install transformers torch soundfile evaluate sentencepiece pydub
!pip install numpy librosa jiwer  fastdtw scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 34.3 MB/s eta 0:00:00
  Created wheel for fastdtw: filename=fastdtw-0.3.4-cp312-cp312-linux_x86_64.whl size=567857 sha256=628d220d290d8a05b69d95700123a348d9723298cca99e4bef0b3938c5bebcd9
  Stored in directory: /root/.cache/pip/wheels/ab/d0/26/b82cb0f49ae73e5e6bba4e8462fff2c9851d7bd2ec64f8891e
Successfully built fastdtw


#2. importation

In [2]:
import os
import soundfile as sf
import numpy as np
import torch
from transformers import pipeline
import evaluate
import re
import unicodedata
import librosa
from jiwer import wer, cer
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan, BarkModel, BarkProcessor, VitsModel, AutoTokenizer
from IPython.display import Audio


#3. Functions


In [3]:
def remove_accents(text):
    """Remove accents: café → cafe"""
    # Normalize to NFD (decompose accents)
    nfd = unicodedata.normalize('NFD', text)
    # Keep only non-accent characters
    return ''.join(char for char in nfd if unicodedata.category(char) != 'Mn')

def normalize_text_for_wer(s):
    # remove accents
    s = remove_accents(s)
    # lowercase
    s = s.lower()
    # remove punctuation and multiple spaces
    s = re.sub(r"[^\w\s]", "", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def save_wav(path, waveform: np.ndarray, sr: int=24000):
    # waveform expected float32 in [-1, 1]
    sf.write(path, waveform.astype(np.float32), sr)
    # Save to WAV
    print(f"✅ Audio saved as ${path}")

# function wer, cer
def calculate_wer(reference: str, hypothesis: str) -> float:
    """Calculates the Word Error Rate (WER) between a reference and a hypothesis string."""
    return wer(reference, hypothesis)

def calculate_cer(reference: str, hypothesis: str) -> float:
    """Calculates the Character Error Rate (CER) between a reference and a hypothesis string."""
    return cer(reference, hypothesis)

def prosody_accuracy_metrics(f0_ref, f0_synth):
    # Both inputs are pitch (F0) arrays aligned in time, with unvoiced frames as 0 or np.nan handled similarly
    mask = (f0_ref > 0) & (f0_synth > 0)  # consider only voiced frames
    f0_ref_voiced = f0_ref[mask]
    f0_synth_voiced = f0_synth[mask]
    if len(f0_ref_voiced) == 0 or len(f0_synth_voiced) == 0:
        return None  # no voiced segments to compare

    rmse = np.sqrt(np.mean((f0_ref_voiced - f0_synth_voiced) ** 2))
    correlation = np.corrcoef(f0_ref_voiced, f0_synth_voiced)[0, 1]
    return rmse, correlation

#4. Load models

In [4]:
# === Load models (example: SpeechT5 and Bark) ===
device = 0 if torch.cuda.is_available() else -1
print("Device (cuda index or -1):", device)

Device (cuda index or -1): -1


##4.1 Model SpeechT5

In [5]:

print("Loading SpeechT5 (this can be slow)...")
# Load models
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

# --- Start of code to load speaker embedding from a local .npy file ---
# Specify the local path to the .npy file you want to use
# Replace 'path/to/your/speaker_embedding.npy' with the actual path to one of your 7 files
local_embedding_path = '/content/cmu_us_awb_arctic-wav-arctic_a0001.npy' # <--- CHANGE THIS PATH

try:
    # Load the embedding from the local .npy file
    speaker_embeddings = torch.tensor(np.load(local_embedding_path)).unsqueeze(0)

    print(f"Successfully loaded speaker embedding from local file: {local_embedding_path}")

except Exception as e:
    print(f"An error occurred while loading the local embedding: {e}")
    # Fallback to a random embedding for other errors
    g = torch.Generator().manual_seed(42)
    speaker_embeddings = torch.randn((1, 512), generator=g)
    print("Using a random speaker embedding as a fallback.")
# --- End of code to load speaker embedding from a local .npy file ---


# Text to synthesize
text = "Welcome to inclusive education number 12 with AI Dr. smith."
inputs = processor(text=text, return_tensors="pt")

# Generate speech
with torch.no_grad():
    speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

Loading SpeechT5 (this can be slow)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/585M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/50.7M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/50.6M [00:00<?, ?B/s]

An error occurred while loading the local embedding: [Errno 2] No such file or directory: '/content/cmu_us_awb_arctic-wav-arctic_a0001.npy'
Using a random speaker embedding as a fallback.


In [6]:
Audio(speech, rate=16000)

In [7]:
save_wav('audio_speechT5.wav', speech.numpy(), sr=16000)


✅ Audio saved as $audio_speechT5.wav


#4.2 Model Bark

In [8]:
model = BarkModel.from_pretrained("suno/bark-small")
processor = BarkProcessor.from_pretrained("suno/bark-small")

# add a speaker embedding
inputs = processor("Welcome to inclusive education number 12 with AI Dr. smith.", voice_preset="v2/en_speaker_3")

# Generate speech
# The output of model.generate is a tensor with shape (batch_size, num_samples)
# We need to convert it to a numpy array and ensure it's in the correct format for saving
with torch.no_grad():
    speech_output_tensor = model.generate(**inputs)

# Convert the tensor to a numpy array and remove the batch dimension
speech_output = speech_output_tensor.cpu().numpy().squeeze()

# Bark model typically outputs raw audio data as float32
# We need to ensure it's in the range [-1, 1] if necessary, although soundfile often handles this.
# For saving with soundfile, a 1D numpy array is usually expected for mono audio.

print("Speech generated with Bark model.")

# The sampling rate for Bark is available in the model's generation_config
sampling_rate_bark = model.generation_config.sample_rate
print(f"Bark model sampling rate: {sampling_rate_bark}")

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

speaker_embeddings_path.json: 0.00B [00:00, ?B/s]

...

speaker_embeddings/v2/en_speaker_3_seman(…):   0%|          | 0.00/3.54k [00:00<?, ?B/s]

speaker_embeddings/v2/en_speaker_3_coars(…):   0%|          | 0.00/10.4k [00:00<?, ?B/s]

speaker_embeddings/v2/en_speaker_3_fine_(…):   0%|          | 0.00/20.6k [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Speech generated with Bark model.
Bark model sampling rate: 24000


In [9]:
Audio(speech_output, rate=sampling_rate_bark)

In [10]:
save_wav('audio_bark_small.wav', speech_output, sr=sampling_rate_bark)


✅ Audio saved as $audio_bark_small.wav


#4.3 MMS-Facebook

In [11]:
model = VitsModel.from_pretrained("facebook/mms-tts-eng")
tokenizer = AutoTokenizer.from_pretrained("facebook/mms-tts-eng")

text = "Welcome to inclusive education number 12 with AI Dr. smith."
inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    output = model(**inputs).waveform

speech_output = output.numpy().squeeze()
sampling_rate = model.config.sampling_rate

print("Speech generated successfully!")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/145M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/413 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/47.0 [00:00<?, ?B/s]

Speech generated successfully!


In [12]:
speech_output

array([-8.2965653e-07, -3.5927540e-05, -2.9583100e-05, ...,
        3.6514593e-05,  1.6763503e-05, -2.3832412e-05], dtype=float32)

In [13]:
Audio(speech_output, rate=sampling_rate)

In [14]:
save_wav('audio_mms_facebook.wav', speech_output, sr=sampling_rate)

✅ Audio saved as $audio_mms_facebook.wav


In [15]:
# --- File Paths (Update as needed) ---
reference_audio_path = 'Welcome to inclusive education number 12 with AI Dr. smith.'
speecht5_audio_path = 'audio_speechT5.wav'
bark_audio_path = 'audio_bark_small.wav'
mms_audio_path = 'audio_mms_facebook.wav'

# --- ASR pipeline for transcription (needed for WER/CER) ---
asr_model_name = "openai/whisper-small"
try:
    asr = pipeline("automatic-speech-recognition", model=asr_model_name, device=0 if torch.cuda.is_available() else -1)
except Exception as e:
    print(f"Error loading ASR pipeline: {e}. WER/CER calculation may be affected.")
    asr = None



config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [16]:
# Define the reference text used for speech generation
reference_text = "Welcome to inclusive education with AI Dr. smith."

# Normalize the reference text for WER/CER calculation
normalized_reference_text = normalize_text_for_wer(reference_text)
print(f"Normalized Reference Text: {normalized_reference_text}")

# List of audio files to transcribe and evaluate
audio_files_to_evaluate = {
    "SpeechT5": speecht5_audio_path,
    "Bark": bark_audio_path,
    "MMS-Facebook": mms_audio_path,
}

# Check if the ASR pipeline was loaded successfully
if asr is not None:
    print("\n--- Transcription and Metric Calculation ---")
    for model_name, audio_path in audio_files_to_evaluate.items():
        print(f"\nEvaluating {model_name} ({audio_path})...")
        try:
            # Check if the audio file exists
            if not os.path.exists(audio_path):
                print(f"Error: Audio file not found at {audio_path}. Skipping.")
                continue

            # Transcribe the audio file
            # The ASR pipeline expects the audio file path or a dictionary with "array" and "sampling_rate"
            # Assuming the ASR model handles the audio loading from the path
            transcription_result = asr(audio_path)
            transcribed_text = transcription_result["text"]
            print(f"Transcribed Text: {transcribed_text}")

            # Normalize the transcribed text
            normalized_transcribed_text = normalize_text_for_wer(transcribed_text)
            print(f"Normalized Transcribed Text: {normalized_transcribed_text}")

            # Calculate WER and CER
            wer_score = calculate_wer(normalized_reference_text, normalized_transcribed_text)
            cer_score = calculate_cer(normalized_reference_text, normalized_transcribed_text)

            print(f"  WER: {wer_score:.4f}")
            print(f"  CER: {cer_score:.4f}")

            # Note about prosody metrics:
            print("  Prosody accuracy metrics (RMSE, Correlation) require pitch (F0) extraction, which is not directly available from the current audio format or ASR output.")


        except Exception as e:
            print(f"An error occurred during evaluation for {model_name}: {e}")

else:
    print("\nASR pipeline not loaded. Cannot perform transcription and metric calculation.")

Normalized Reference Text: welcome to inclusive education with ai dr smith

--- Transcription and Metric Calculation ---

Evaluating SpeechT5 (audio_speechT5.wav)...


`return_token_timestamps` is deprecated for WhisperFeatureExtractor and will be removed in Transformers v5. Use `return_attention_mask` instead, as the number of frames can be inferred from it.
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.


Transcribed Text:  Welcome to Inclusive Education Number with I. Dr. Smith.
Normalized Transcribed Text: welcome to inclusive education number with i dr smith
  WER: 0.2500
  CER: 0.1702
  Prosody accuracy metrics (RMSE, Correlation) require pitch (F0) extraction, which is not directly available from the current audio format or ASR output.

Evaluating Bark (audio_bark_small.wav)...
Transcribed Text:  Welcome to inclusive education number 12 with AI Dr. Smith.
Normalized Transcribed Text: welcome to inclusive education number 12 with ai dr smith
  WER: 0.2500
  CER: 0.2128
  Prosody accuracy metrics (RMSE, Correlation) require pitch (F0) extraction, which is not directly available from the current audio format or ASR output.

Evaluating MMS-Facebook (audio_mms_facebook.wav)...
Transcribed Text:  Welcome to Inclusive Education Number Poll with Hader Smith.
Normalized Transcribed Text: welcome to inclusive education number poll with hader smith
  WER: 0.5000
  CER: 0.3404
  Prosody accura